### 데이터 타입 기록###
A. 데이터 설명

본 대회에서는 한국수력원자력(주)에서 제공받은 원자력 발전소 모사 데이터 및 실제 데이터가 제공됩니다. 

주어진 10분 사이에 발전소의 상태가 변하며 변하기 전 발전소 상태를 상태_A, 변한 후 상태를 상태_B라고 할 때 

데이터를 기반으로 상태_B를 판단하는 것이 대회의 목적입니다. 모든 데이터는 같은 상태 (상태_A)에서 시작합니다. 



모사 데이터는 0초에서 15초 사이에 상태가 변하기 시작합니다. 모든 데이터가 숫자로 이루어져 있습니다. 

반면 실제 데이터는 10초에 상태가 변하기 시작합니다. 실제 데이터에는 Bad, CLOSE, Equip Fail, No Data, Normal, OFF, ON, OPEN, System.Char[] 등 문자열 데이터를 포함하고 있으며 결측치가 있을 수 있습니다. 



훈련 데이터로는 10분 데이터가 주어지며 테스트 데이터로는 1분 간 데이터가 주어집니다. 즉 1분 데이터만 사용하여 평가합니다. 



B. 파일 설명

1. train.zip(9.81GB) : 훈련 데이터가 포함된 zip 파일입니다.

2. test.zip(879MB) : 테스트 데이터가 포함된 zip 파일입니다.

   train.zip, test.zip 내 파일이름은 해당되는 고유한 번호 (id) 로 되어 있습니다. train과 test의 id는 중복되지 않습니다.

3. train_label.csv(6.78KB) : 훈련데이터의 id (파일 이름)에 해당되는 상태_B를 정리한 파일입니다. 

4. sample_submission.csv(283KB) : 제출 형식과 동일한 샘플 데이터 입니다.

5. additional_data.zip(13.9MB) : 실제 발전소 운전 데이터 입니다. 참고자료로 주어지는 파일입니다. (컬럼은 train 및 test 데이터와 같습니다.)



데이터 파일 컬럼에 대한 설명은 아래와 같습니다.

time : 발전소 운전 경과 시간 입니다. 단위는 초 입니다.

V0000~V5120 : 발전소 운영 중 측정되는 변수 입니다. 비식별화 되어 제공됩니다.

O 코드 관련

1) 입상자는 코드 제출 필수. 제출 코드는 예측 결과를 리더보드 점수로 복원할 수 있어야 함

2) 코드 제출시 확장자가 R user는 R or .rmd. Python user는 .py or .ipynb

3) 코드에 ‘/data’ 데이터 입/출력 경로 포함 제출

4) 전체 프로세스를 일목요연하게 정리하여 주석을 포함하여 하나의 파일로 제출

5) 모든 코드는 오류 없이 실행되어야 함(라이브러리 로딩 코드 포함되어야 함).

6) 코드와 주석의 인코딩은 모두 UTF-8을 사용하여야 함 

## metric
평가 지표는 Log Loss를 사용합니다.

In [1]:
import numpy as np
import pandas as pd

from NU_utils import load_eda_statistic_pd,reduce_mem_usage

import os
import gc

import matplotlib.pyplot as plt
import seaborn as sns

import dill as pickle

from catboost import Pool

In [2]:
train = pd.read_csv("data/train/0.csv")
target = pd.read_csv("data/train_label.csv")
test = pd.read_csv("data/test/828.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")
print("{} observations and {} features in {} train set.".format(train.shape[0],train.shape[1], len(os.listdir("data/train"))))
print("{} observations and {} features in {} test set.".format(test.shape[0],test.shape[1],len(os.listdir("data/test"))))
print("train : test row ratio {} and file {}".format(train.shape[0] / test.shape[0], len(os.listdir("data/train"))/len(os.listdir("data/test"))))

600 observations and 5122 features in 828 train set.
60 observations and 5122 features in 720 test set.
train : test row ratio 10.0 and file 1.15


### Issue:
- column 수 많음 -> 과적합
- train 을 모델에 어떻게 넣을 것인지  
ex1) train 파일 827개 붙이기  
ex2) 600 개마다 압축해서 1개씩 row추출 -> new dataset 만들기 shape(827, 1)  
- train, test 분포 같은지 확인  


In [3]:
target.head()

,id,label
0,0,110
1,1,29
2,2,111
3,3,176
4,4,130


In [4]:
train.head()

,time,V0000,V0001,V0002,V0003,V0004,V0005,V0006,V0007,V0008,...,V5111,V5112,V5113,V5114,V5115,V5116,V5117,V5118,V5119,V5120
0,0,30.486806,8.682230,8.736399,8.691251,8.711798,204.576461,201.196818,-1.213967e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,127.58,112.263,-0.264703,85.4,-0.004474
1,1,30.460030,8.784153,8.691244,8.706254,8.716143,194.222395,169.979333,-2.279681e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,0.000014,85.4,0.000000
2,2,30.475103,8.812022,8.697733,8.721917,8.681361,193.078139,160.719360,-1.586760e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,-0.000013,85.4,0.000000
3,3,30.455413,8.908323,8.712004,8.723829,8.724655,207.644178,232.675873,-4.016423e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,0.000019,85.4,0.000000
4,4,30.472642,8.815620,8.702882,8.722939,8.702341,186.112451,160.829340,8.676680e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,-0.000008,85.4,0.000000


### 정리
- 5121개의 column이 Anonymized 된 binary classifiacation, multivarate timeseries  
모든 변수가 timeseries인 상태, iid 기반 모델 사용불가, 중요 변수를 어떻게 선택할지
- 5122개의 column / 600 * 827 개의 row 
- train , test 는 모사데이터 => 인공 데이터인경우 정규분포 기반 모델 잘통함 QDA, GMM 등등

In [5]:
train.info()
print("\n")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Columns: 5122 entries, time to V5120
dtypes: float64(5121), int64(1)
memory usage: 23.4 MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Columns: 5122 entries, time to V5120
dtypes: float64(5121), int64(1)
memory usage: 2.3 MB


- int 인 변수들도 float으로 되어 있음 => 정리 필요

In [6]:
eda = load_eda_statistic_pd("eda.pkl",train)

In [7]:
eda["isnull_table"].sum(axis=0)

0

- 모든 column 결측치 없음

In [8]:
train,_ = reduce_mem_usage(train,False)

Memory usage of properties dataframe is : 23.44677734375MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  7.11376953125  MB
This is  30.340073720818843 % of the initial size


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Columns: 5122 entries, time to V5120
dtypes: float32(2434), int8(23), uint16(8), uint8(2657)
memory usage: 7.1 MB



- 소수 : 32개
- -127 ~ 128 : 23개
- 128 이상 자연수 : 8개
- 0 ~ 255 : 2657개

### 0번 데이터셋 eda
상태 A : 0 , 상태 B : 1 로 Y 값을 training set 에 삽입 후 line chart

In [10]:
def insert_label(train, label, n = 0):
    """
    train dataset 1개(600 rows) 에 0,1 label 삽입하는 함수
    
    args :
        n -- (int) nth training set
        train -- (pd.DataFrame) one training set , shape (600,5122)
        label -- (pd.DataFrame) Y label, shape (828, 1)
    return
        merged_train -- (pd.DataFrame) label added train set, shape (600, 5123)
    """
    merged_train = train.copy()
    B = int(label["label"][label['id'] == n])
    y = np.zeros(len(train))
    y[B:] = 1
    merged_train["target"]  = y
    return merged_train

In [11]:
merged_train = insert_label(train,target)

### Line chart 정리
- 변화가 미미한 feature :ex)V007, 8, 10, 12...
- 상태 변화시에 두드러지는 등락을 보이는 feature :ex)163,164,165,166..
- 증가추세를 보이는 feature :ex) 130,169,179 ...
- 5122 개의 column 그래프는 너무 많으므로 파일 한곳에 png 저장 필요

### 변수선택 방법
- variance treshold? pca? 
- 시계열로 보지 말고 y를 각 상태에서의 binary classification으로 보고 iid 모델 적용해도 되지 않을까?
    - 각 변수들은 초기 상태에 의존??
        - 의존한다면 test set 파일마다 비슷한 분포의 training set을 골라서 그 training set만 학습, 적용
        - 혹은 초기상태 값을 모두 파일마다 뺀다음에 training?
        => 초기 상태에 의존한다는 점을 어떻게 알 수 있을까?
- 인공데이터인점을 이용할 수 없을까? 
    - normal 분포 활용 방안?
    
- LANL 대회 참고하기
    - 유사점 : 그 대회는 마지막 obs에서 지진까지 남은 시간을 예측하는 대회  
    따라서 여러개의 row를 가진 여러개의 train set을 각각 하나의 예측값을 내는 것이 유사
    - LANL 대회처럼 상태 B까지 남은 시간을 예측하는 식으로 모델링?

### Y 모델링 방법
1. train label이 110 이면 110번째부터 데이터를 True, 이전까지는 False로 두고 binary classification
2. 상태변화가 일어나기까지 시간을 Y로 둠 train label이 110이면 time 변수가 1 인 데이터는 Y를 109, time 변수가 2인 데이터는 Y를 108.. time 변수가 300인 데이터는 -190 등등
3. RNN 모형으로 한번에 모델링하여 마지막에 output을 한개만 내는 방식으로 training

=> 1,2 일단 둘다 해보기

In [12]:
os.listdir("data/train")

['0.csv',
 '1.csv',
 '10.csv',
 '100.csv',
 '101.csv',
 '102.csv',
 '103.csv',
 '104.csv',
 '105.csv',
 '106.csv',
 '107.csv',
 '108.csv',
 '109.csv',
 '11.csv',
 '110.csv',
 '111.csv',
 '112.csv',
 '113.csv',
 '114.csv',
 '115.csv',
 '116.csv',
 '117.csv',
 '118.csv',
 '119.csv',
 '12.csv',
 '120.csv',
 '121.csv',
 '122.csv',
 '123.csv',
 '124.csv',
 '125.csv',
 '126.csv',
 '127.csv',
 '128.csv',
 '129.csv',
 '13.csv',
 '130.csv',
 '131.csv',
 '132.csv',
 '133.csv',
 '134.csv',
 '135.csv',
 '136.csv',
 '137.csv',
 '138.csv',
 '139.csv',
 '14.csv',
 '140.csv',
 '141.csv',
 '142.csv',
 '143.csv',
 '144.csv',
 '145.csv',
 '146.csv',
 '147.csv',
 '148.csv',
 '149.csv',
 '15.csv',
 '150.csv',
 '151.csv',
 '152.csv',
 '153.csv',
 '154.csv',
 '155.csv',
 '156.csv',
 '157.csv',
 '158.csv',
 '159.csv',
 '16.csv',
 '160.csv',
 '161.csv',
 '162.csv',
 '163.csv',
 '164.csv',
 '165.csv',
 '166.csv',
 '167.csv',
 '168.csv',
 '169.csv',
 '17.csv',
 '170.csv',
 '171.csv',
 '172.csv',
 '173.csv',
 '17

In [13]:
#for i in range(10):
    #train = pd.read_csv("data/train/" + i + ".csv")
    #merged_train = insert_label(train,target)
    #features = train.columns[1:65]
    
    #plt.savefig(filename)#

In [16]:
from sklearn.model_selection import train_test_split

### 모델링을 위해 subdata 만들기
1. 빠르면서도 전체 데이터를 잘 반영해야함
2. 827개 데이터니까 80개마다 뽑아서 10개 데이터를 사용해서 해보자

In [21]:
candidate = [0,80,160,240,320,400,480,560]
subtrain = []
subtarget = target.iloc[candidate,1]
for i in [0,80,160,240,320,400,480,560]:
    _train = pd.read_csv("data/train/" + str(i) + ".csv")
    subtrain.append(_train)

In [22]:
subtarget

0      110
80     118
160    123
240    189
320    128
400     82
480    172
560    165
Name: label, dtype: int64

In [18]:
SEED = 42

[     time      V0000     V0001     V0002     V0003     V0004       V0005  \
 0       0  30.486806  8.682230  8.736399  8.691251  8.711798  204.576461   
 1       1  30.460030  8.784153  8.691244  8.706254  8.716143  194.222395   
 2       2  30.475103  8.812022  8.697733  8.721917  8.681361  193.078139   
 3       3  30.455413  8.908323  8.712004  8.723829  8.724655  207.644178   
 4       4  30.472642  8.815620  8.702882  8.722939  8.702341  186.112451   
 ..    ...        ...       ...       ...       ...       ...         ...   
 595   595  30.483134  8.575579  8.714590  8.651008  8.722999  183.323223   
 596   596  30.450394  8.769416  8.684979  8.612432  8.726384  218.315848   
 597   597  30.462762  8.685032  8.673951  8.687693  8.692903  203.192111   
 598   598  30.471073  8.702494  8.713313  8.774587  8.682366  196.369794   
 599   599  30.456627  8.733662  8.692675  8.705273  8.714962  200.779053   
 
           V0006         V0007  V0008  ...  V5111  V5112  V5113  V5114  V5

## 1번 방향 starter

In [24]:
subtrain[0]
#에서 time변수빼고

,time,V0000,V0001,V0002,V0003,V0004,V0005,V0006,V0007,V0008,...,V5111,V5112,V5113,V5114,V5115,V5116,V5117,V5118,V5119,V5120
0,0,30.486806,8.682230,8.736399,8.691251,8.711798,204.576461,201.196818,-1.213967e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,127.58,112.263,-0.264703,85.4,-0.004474
1,1,30.460030,8.784153,8.691244,8.706254,8.716143,194.222395,169.979333,-2.279681e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,0.000014,85.4,0.000000
2,2,30.475103,8.812022,8.697733,8.721917,8.681361,193.078139,160.719360,-1.586760e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,-0.000013,85.4,0.000000
3,3,30.455413,8.908323,8.712004,8.723829,8.724655,207.644178,232.675873,-4.016423e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,0.000019,85.4,0.000000
4,4,30.472642,8.815620,8.702882,8.722939,8.702341,186.112451,160.829340,8.676680e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,-0.000008,85.4,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,595,30.483134,8.575579,8.714590,8.651008,8.722999,183.323223,211.530830,1.435986e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,-0.000011,85.4,0.000000
596,596,30.450394,8.769416,8.684979,8.612432,8.726384,218.315848,185.905455,-3.382820e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,0.000001,85.4,0.000000
597,597,30.462762,8.685032,8.673951,8.687693,8.692903,203.192111,193.775149,-9.662521e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,-0.000024,85.4,0.000000
598,598,30.471073,8.702494,8.713313,8.774587,8.682366,196.369794,199.071209,-4.709846e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,0.000025,85.4,0.000000


In [ ]:
%%time

#Starter 모델 만들기
#빠르게 학습, 테스트 위해 하이퍼파라미터 설정
# 1분 내로 줄이면서 KFold 스코어랑 많이 차이 안나게
features = [c for c in train.columns if c not in ['id', 'target']]
target = train['target']
X_train, X_valid, y_train, y_valid = train_test_split(train[features], target, test_size=0.33, random_state=42)

trn_data = Pool(X_train, y_train)
val_data = Pool(X_valid, y_valid)

model = CatBoostClassifier(task_type = "GPU",
                           devices='0',
                           random_seed = 10)

fit_model = model.fit(trn_data,
                     eval_set = val_data,
                     use_best_model = True,
                     verbose = 0,
                     plot = False)

p_valid = fit_model.predict_proba(X_valid[features].values)[:,1]
auc = roc_auc_score(y_valid, p_valid)
print(f'AUC: {auc:.5}')